In [3]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# モデルごとにグループ化して集計
grouped_df = combined_df.groupby('Model', sort=False).agg({
    'Average Test Score': ['mean', 'std']
}).reset_index()

# カラム名のリネーム
grouped_df.columns = ['Model', 'Average Test Score Mean', 'Average Test Score Std']

# 小数点2位までに丸める
grouped_df['Average Test Score Mean'] = grouped_df['Average Test Score Mean'].round(2)
grouped_df['Average Test Score Std'] = grouped_df['Average Test Score Std'].round(2)

# モデルごとにデータを分割
rbf_df = grouped_df[grouped_df['Model'].str.startswith('RBF')]
linear_df = grouped_df[grouped_df['Model'].str.startswith('LINEAR')]
old_df = grouped_df[grouped_df['Model'].str.startswith('OLD')]

# 各データフレームのインデックスをリセット
rbf_df.reset_index(drop=True, inplace=True)
linear_df.reset_index(drop=True, inplace=True)
old_df.reset_index(drop=True, inplace=True)

# データを横に並べる
final_df = pd.concat([rbf_df, linear_df, old_df], axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



      Model  Average Test Score Mean  Average Test Score Std        Model  \
0  RBF_LQP1                    88.82                    4.40  LINEAR_LQP1   
1   RBF_SQP                    78.91                    5.36   LINEAR_SQP   
2  RBF_LQP2                    61.18                    3.87  LINEAR_LQP2   

   Average Test Score Mean  Average Test Score Std     Model  \
0                    87.46                    5.74  OLD_LQP1   
1                    78.13                    7.32   OLD_SQP   
2                    62.00                    5.23  OLD_LQP2   

   Average Test Score Mean  Average Test Score Std  
0                    87.22                    1.58  
1                    50.78                    0.96  
2                    56.98                    1.61  


In [4]:
import pandas as pd

# すべてのCSVファイル名をリストにまとめる
csv_files = [f'statistics_data{i}.csv' for i in range(1, 11)]

# 各CSVファイルを読み込み、データフレームをリストに保存
dataframes = [pd.read_csv(file) for file in csv_files]

# すべてのデータフレームを縦に連結
combined_df = pd.concat(dataframes, ignore_index=True)

# Modelの順序を保持するためにカテゴリー型に変更
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=combined_df['Model'].unique(), ordered=True)

# 必要なカラムを選択
selected_columns = ['Model', 'Average TNR', 'Average TPR', 'Average Test Score']
combined_df = combined_df[selected_columns]

# モデルごとにデータを分割してリストに保存
models = combined_df['Model'].unique()
model_dfs = [combined_df[combined_df['Model'] == model].reset_index(drop=True) for model in models]

# 各モデルごとにデータフレームを作成し、リストに追加
final_dfs = []
for df in model_dfs:
    model_name = df['Model'].iloc[0]
    avg_tnr_mean = round(df['Average TNR'].mean(), 2)
    avg_tpr_mean = round(df['Average TPR'].mean(), 2)
    avg_test_score_mean = round(df['Average Test Score'].mean(), 2)
    avg_test_score_std = round(df['Average Test Score'].std(), 2)
    
    summary = pd.DataFrame({
        'Model': [model_name + '_MEAN'],
        'Average TNR': [avg_tnr_mean],
        'Average TPR': [avg_tpr_mean],
        'Average Test Score': [avg_test_score_mean],
        'Average Test Score Std': [avg_test_score_std]
    })
    
    df['Average Test Score Std'] = pd.NA  # 標準偏差の列を追加してNaNで埋める
    summary['Average Test Score Std'] = avg_test_score_std  # 標準偏差の列を更新
    
    df = df.drop(columns=['Model'])
    df.columns = [f'{model_name} {col}' for col in df.columns]
    
    summary.columns = [f'{model_name} {col}' for col in summary.columns]
    
    df = pd.concat([df, summary], ignore_index=True)
    
    final_dfs.append(df)

# すべてのモデルのデータフレームを横に連結
final_df = pd.concat(final_dfs, axis=1)

# 結果を表示
print(final_df)

# CSVファイルとして保存
final_df.to_csv('grouped_statistics.csv', index=False)



    RBF_LQP1 Average TNR  RBF_LQP1 Average TPR  RBF_LQP1 Average Test Score  \
0                  93.52                 95.22                        94.37   
1                  77.85                 98.22                        88.04   
2                  77.37                 97.04                        87.20   
3                  85.93                 97.56                        91.74   
4                  90.30                 94.07                        92.19   
5                  67.52                 97.22                        82.37   
6                  66.70                 98.15                        82.43   
7                  73.85                 96.63                        85.24   
8                  92.30                 91.67                        91.98   
9                  89.00                 96.37                        92.69   
10                 81.43                 96.22                        88.83   

   RBF_LQP1 Average Test Score Std RBF_LQP1 Model  